In [1]:
# %%writefile train.py
import os
# Loading the iconic trio 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
plt.style.use('fivethirtyeight')

# Importing model_selection to get access to some dope functions like GridSearchCV()
from sklearn import model_selection

from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SVMSMOTE

# Loading models
from sklearn import tree
from sklearn import ensemble
#import xgboost
from sklearn import linear_model

# custom
#from custom import helper

# Loading evaluation metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

import pickle

from pycaret.classification import * 

from sklearn.model_selection import train_test_split

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

In [3]:
current_directory = os.getcwd()

# Print the current working directory
print("Current Working Directory:", current_directory)

Current Working Directory: G:\Semester Arbeit\Programming\notebooks


In [107]:
healthy_df = pd.read_csv("./healthy/x24_03_25_15_32_55_112_5_500_Nm_1000_rpm_features.csv",sep=",")

In [108]:
healthy_df_2 = pd.read_csv("./healthy/x24_03_25_15_58_41_112_5_500_Nm_1900_2000_rpm_features.csv",sep=",")

In [109]:
healthy_df_3 = pd.read_csv("./healthy/x24_03_25_16_16_27_112_5_500_Nm_3000_rpm_features.csv",sep=",")

In [161]:
#healthy_df.head()

In [111]:
damaged_df = pd.read_csv("./damaged/x24_06_10_15_16_00_112_5_500_Nm_3000_rpm_GF_Ritzel_features.csv",sep=",")

In [162]:
#damaged_df.head()

In [113]:
damaged_df_2 = pd.read_csv("./damaged/x24_06_10_15_13_05_112_5_500_Nm_2000_rpm_GF_Ritzel_features.csv",sep=",")

In [114]:
damaged_df_3 = pd.read_csv("./damaged/x24_06_10_15_09_55_112_5_500_Nm_1000_rpm_GF_Ritzel_features.csv",sep=",")

In [175]:
combined_df_unshuffled = pd.concat([healthy_df, damaged_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df = combined_df_unshuffled.sample(frac=1, random_state=42).reset_index(drop=True)


In [176]:
len(combined_df)

17998

In [177]:
combined_df.head(-20)

,Date,Time,Torque,Velocity,Label,mean_ch1,rms_ch1,std_dev_ch1,skewness_ch1,kurtosis_ch1,...,spectral_skewness_ch5,spectral_kurtosis_ch5,mean_wavelet_ch5,var_wavelet_ch5,entropy_wavelet_ch5,energy_wavelet_ch5,mean_spectrogram_ch5,var_spectrogram_ch5,entropy_spectrogram_ch5,energy_spectrogram_ch5
0,NaN,NaN,NaN,NaN,damaged,0.004508,0.003464,1.201130,-0.001533,0.003492,...,0.410191,0.774846,-1.307014e+03,1.603819e-05,6.149238e-06,5.330387e-02,9.351872e-02,5.111322e+02,8.344249e-07,2.953718e-07
1,NaN,NaN,NaN,NaN,healthy,0.001252,0.003170,2.478650,-0.002384,0.003345,...,0.000030,0.000007,-7.382575e+03,4.914173e-09,6.786000e-09,1.621505e-06,3.337609e-07,2.001033e+03,1.762575e-10,2.486874e-10
2,NaN,NaN,NaN,NaN,healthy,0.011747,-0.006827,2.639480,-0.002084,0.003300,...,0.000016,0.000007,-7.962324e+03,2.475487e-09,5.091740e-09,8.439671e-07,3.590412e-07,2.160134e+03,8.484665e-11,1.826470e-10
3,NaN,NaN,NaN,NaN,damaged,0.014410,0.008257,1.219658,-0.001589,0.004534,...,0.203084,1.248840,-1.362145e+03,2.019172e-05,8.904019e-06,2.452397e-02,1.757773e-01,5.269688e+02,1.081848e-06,4.360412e-07
4,NaN,NaN,NaN,NaN,healthy,-0.005833,0.011813,2.491649,-0.002937,0.002789,...,0.000036,0.000040,-7.409937e+03,1.725852e-08,3.399866e-09,1.917265e-06,2.208213e-06,2.018023e+03,6.556993e-10,1.218101e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17973,NaN,NaN,NaN,NaN,healthy,0.012466,-0.003102,0.002578,-0.001947,0.003330,...,0.000025,0.000014,3.280961e-08,6.041074e-09,6.359846e-09,1.294264e-06,7.359823e-07,1.292319e-09,2.166231e-10,2.275577e-10
17974,NaN,NaN,NaN,NaN,healthy,-0.000330,0.004928,2.472216,-0.002137,0.002609,...,0.000029,0.000010,-7.338703e+03,5.936291e-09,3.330134e-09,1.610093e-06,5.274749e-07,1.995938e+03,2.152817e-10,1.173105e-10
17975,NaN,NaN,NaN,NaN,healthy,0.005375,-0.002811,-0.003507,-0.001683,0.002987,...,0.000060,0.000002,2.247759e-08,2.090518e-09,3.309693e-09,3.463973e-06,8.915988e-08,8.776376e-10,7.130869e-11,1.169785e-10
17976,NaN,NaN,NaN,NaN,healthy,0.000529,0.006243,0.476460,-0.003007,0.002825,...,0.000009,0.000003,1.723010e+00,7.801909e-09,4.571894e-09,4.490282e-07,1.255154e-07,3.130845e+00,2.856380e-10,1.642871e-10


In [178]:
categories = combined_df.columns.to_list()
#print(categories)

In [179]:
features = categories[5:]
#print(features)

In [180]:
experiment = setup(combined_df, target="Label", categorical_features=features)

,Description,Value
0,Session id,6485
1,Target,Label
2,Target type,Binary
3,Target mapping,"damaged: 0, healthy: 1"
4,Original data shape,"(17998, 145)"
5,Transformed data shape,"(17998, 141)"
6,Transformed train set shape,"(12598, 141)"
7,Transformed test set shape,"(5400, 141)"
8,Numeric features,4
9,Categorical features,140


In [181]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.6790
knn,K Neighbors Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.3500
nb,Naive Bayes,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0840
dt,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0990
svm,SVM - Linear Kernel,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0870
ridge,Ridge Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1020
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.2230
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1200
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,2.9140
lda,Linear Discriminant Analysis,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1160


In [182]:
evaluate_model(best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [183]:
predict_model(best_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


,Date,Time,Torque,Velocity,mean_ch1,rms_ch1,std_dev_ch1,skewness_ch1,kurtosis_ch1,peak_to_peak_ch1,...,var_wavelet_ch5,entropy_wavelet_ch5,energy_wavelet_ch5,mean_spectrogram_ch5,var_spectrogram_ch5,entropy_spectrogram_ch5,energy_spectrogram_ch5,Label,prediction_label,prediction_score
1524,NaN,NaN,NaN,NaN,-0.009767,0.010590,1.222551,-0.002947,0.001438,0.452393,...,4.262937e-06,1.800436e-06,3.685757e-02,9.864379e-02,527.357300,2.078375e-07,8.277645e-08,damaged,damaged,1.0
2770,NaN,NaN,NaN,NaN,0.005533,0.010599,1.128779,-0.002000,0.002892,0.537248,...,1.673176e-05,7.715240e-06,8.202289e-02,1.592772e-01,213.182770,8.724570e-07,3.781517e-07,damaged,damaged,1.0
15161,NaN,NaN,NaN,NaN,0.011383,-0.000198,1.758949,-0.002378,0.002837,0.039678,...,8.871405e-09,4.730708e-09,1.175460e-06,7.868003e-07,974.901978,3.256413e-10,1.718639e-10,healthy,healthy,1.0
8447,NaN,NaN,NaN,NaN,0.002227,0.003440,2.086177,-0.002286,0.003232,0.037671,...,5.195838e-09,5.123479e-09,1.754449e-06,2.866216e-07,1288.817871,1.884349e-10,1.869974e-10,healthy,healthy,1.0
17475,NaN,NaN,NaN,NaN,0.011917,-0.006578,2.654213,-0.001993,0.003231,0.036385,...,6.611838e-09,5.221914e-09,9.596803e-07,3.337487e-07,2179.041016,2.394719e-10,1.882354e-10,healthy,healthy,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6852,NaN,NaN,NaN,NaN,-0.002137,0.005038,0.279307,-0.003403,0.002279,0.033758,...,9.867248e-09,2.580015e-09,9.935578e-07,3.280175e-08,0.036546,3.682599e-10,9.125230e-11,healthy,healthy,1.0
1313,NaN,NaN,NaN,NaN,-0.002227,-0.001445,1.135830,-0.003568,0.001530,0.565703,...,1.730171e-05,4.201387e-06,9.588547e-02,8.315153e-02,459.384399,9.024874e-07,1.977807e-07,damaged,damaged,1.0
12987,NaN,NaN,NaN,NaN,-0.002180,-0.018110,1.227773,-0.003507,0.001115,0.487801,...,8.160157e-06,2.280986e-06,5.637857e-02,1.256602e-01,528.936401,4.098060e-07,1.061112e-07,damaged,damaged,1.0
15239,NaN,NaN,NaN,NaN,-0.000409,0.030514,1.285732,-0.002920,0.002178,0.389508,...,3.867263e-06,3.140575e-06,1.676012e-02,8.394427e-02,280.154205,1.866852e-07,1.494225e-07,damaged,damaged,1.0


In [184]:
X = combined_df.drop(['Date','Time', 'Velocity','Torque', 'Label'], axis=1)

In [185]:
#X.head()

In [186]:
y = combined_df["Label"]

In [187]:
#y

In [188]:
scaler = StandardScaler()

In [189]:
X_scaled = scaler.fit_transform(X)

In [190]:
y_encoded = []

In [191]:
for i in range(len(y)):
    if y[i] == "healthy":
        y_encoded.append(1)
    else:
        y_encoded.append(0)

In [192]:
#y_encoded[-5:]

In [193]:
type(y_encoded)

list

In [194]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [195]:
#X_train

In [196]:
type(y_train)

list

In [197]:
df = X_train

In [198]:
first_row = df.iloc[0]
print("First row as Series:")
print(first_row)

first_row_df = df.iloc[[0]]
print("\nFirst row as DataFrame:")
print(first_row_df)

first_row_values = df.iloc[0].tolist()
print("\nFirst row values as list:")
print(first_row_values)

First row as Series:
mean_ch1                   1.416748e-02
rms_ch1                   -6.488037e-03
std_dev_ch1                2.656000e+00
skewness_ch1              -1.918945e-03
kurtosis_ch1               3.060303e-03
                               ...     
energy_wavelet_ch5         9.910909e-07
mean_spectrogram_ch5       4.608653e-07
var_spectrogram_ch5        2.180795e+03
entropy_spectrogram_ch5    2.609875e-10
energy_spectrogram_ch5     1.715816e-10
Name: 15760, Length: 140, dtype: float64

First row as DataFrame:
       mean_ch1   rms_ch1  std_dev_ch1  skewness_ch1  kurtosis_ch1  \
15760  0.014167 -0.006488        2.656     -0.001919       0.00306   

       peak_to_peak_ch1  crest_factor_ch1  impulse_factor_ch1  \
15760          0.035906          0.028899            4.927548   

       clearance_factor_ch1  shape_factor_ch1  ...  spectral_skewness_ch5  \
15760              0.005234          0.004711  ...               0.000019   

       spectral_kurtosis_ch5  mean_wavelet_ch5

In [199]:
X_train_np = X_train.to_numpy()
y_train_np = y_train
X_test_np = X_test.to_numpy()
y_test_np = y_test

In [200]:
#y_test_np

In [201]:
#X_train_np

In [202]:
train_data = TensorDataset(torch.FloatTensor(X_train_np), torch.FloatTensor(y_train_np))
test_data = TensorDataset(torch.FloatTensor(X_test_np), torch.FloatTensor(y_test_np))

In [203]:
class NN_3(nn.Module):
    def __init__(self):
        super(NN_3, self).__init__()
        self.fc1 = nn.Linear(len(X.columns), 30)
        self.fc2 = nn.Linear(30,30)
        self.fc3 = nn.Linear(30,10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [204]:
model = NN_3()

In [205]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [206]:
train_data = TensorDataset(torch.FloatTensor(X_train_np), torch.LongTensor(y_train))
test_data = TensorDataset(torch.FloatTensor(X_test_np), torch.LongTensor(y_test))

In [207]:
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=32, shuffle=False)

In [208]:
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 0.0000
Epoch [2/10], Loss: 0.0000
Epoch [3/10], Loss: 0.0000
Epoch [4/10], Loss: 0.0000
Epoch [5/10], Loss: 0.0000
Epoch [6/10], Loss: 0.0000
Epoch [7/10], Loss: 0.0000
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0000


In [209]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        #print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test data: {100 * correct / total}%')

Accuracy of the model on the test data: 100.0%


In [210]:
from sklearn.metrics import classification_report

In [211]:
len(predicted)

16

In [212]:
type(test_loader)

torch.utils.data.dataloader.DataLoader

In [213]:
for batch_idx, (data, target) in enumerate(test_loader):
    print(f"Batch {batch_idx+1}")
    print(f"Data: {data}")
    print(f"Target: {target}")

Batch 1
Data: tensor([[-6.2292e-03,  1.5480e-02,  2.5730e+00,  ...,  2.0973e+03,
          5.3851e-10,  8.6068e-11],
        [ 3.5999e-03,  4.4409e-03,  6.3729e-01,  ...,  2.7608e+01,
          2.0982e-10,  2.1005e-10],
        [-4.0747e-03, -3.5754e-02,  1.1634e+00,  ...,  4.8099e+02,
          7.5296e-07,  1.3815e-07],
        ...,
        [-3.8159e-03,  1.6750e-02,  2.5639e+00,  ...,  1.8742e+03,
          4.7279e-10,  7.8273e-11],
        [ 6.0925e-03,  2.5730e-02,  1.2865e+00,  ...,  5.7553e+02,
          3.1973e-07,  2.3804e-07],
        [ 7.7332e-03,  5.1614e-02,  1.2804e+00,  ...,  5.7221e+02,
          1.6332e-07,  2.0699e-07]])
Target: tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
        1, 0, 0, 1, 0, 1, 0, 0])
Batch 2
Data: tensor([[ 8.3203e-03,  2.0422e-03,  1.2952e+00,  ...,  4.3741e+02,
          1.6148e-07,  1.6343e-07],
        [ 1.1503e-02,  2.8843e-02,  1.2123e+00,  ...,  5.1795e+02,
          6.8337e-07,  3.3979e-07],
        [-3.5

In [214]:
from sklearn.neural_network import MLPClassifier

In [215]:
nn_model = MLPClassifier(hidden_layer_sizes=(30,50))

In [216]:
nn_model.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(30, 50), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [217]:
nn_predicted = nn_model.predict(X_test)

In [218]:
from sklearn.metrics import classification_report

In [219]:
report = classification_report(y_test, nn_predicted)

In [220]:
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1816
           1       1.00      1.00      1.00      1784

    accuracy                           1.00      3600
   macro avg       1.00      1.00      1.00      3600
weighted avg       1.00      1.00      1.00      3600

